## TASK-A

linear perceptron

In [74]:
import pandas as pd
import numpy as np

In [75]:
data1 = pd.read_csv('Q1_data.csv') #reading the csv file
data1.head()

,x,y,label
0,-10.000000,-13.471895,0
1,-9.924242,-16.048170,0
2,-9.848485,-14.739494,0
3,-9.772727,-12.063668,0
4,-9.696970,-12.658823,0


In [76]:
data1['label'].unique() #2 class categorical model

array([0, 1])

In [86]:
input_train = data1.iloc[:310,:-1]
target_train = data1.iloc[:310,-1]
input_test = data1.iloc[310:,:-1]
target_test = data1.iloc[310:,-1]

In [87]:
rows = input_train.shape[0]

In [88]:
# heaviside activation function
def activation(z):
    return np.heaviside(z, 0) # haviside(z) heaviside -> activation

In [89]:
def fit(X, y, rate,epochs):
  n_features = X.shape[1]

  # Initializing weights and bias
  weights = np.zeros((n_features))
  bias = 0

  # Iterating until the number of epochs
  for epoch in range(epochs):

    # Traversing through the entire training set
    for i in range(len(X)):
      z = np.dot(X, weights) + bias # Finding the dot product and adding the bias
      y_pred = activation(z) # Passing through an activation function

      #Updating weights and bias
      weights = weights + rate * (y[i] - y_pred[i]) * X[i]
      bias = bias + rate * (y[i] - y_pred[i])

  return weights, bias

In [90]:
def predict(w,b,X):
  z = np.dot(X, w) + b
  return activation(z)

In [92]:
w,b = fit(input_train, target_train, 0.001, 100)

pred = predict(w,b,input_test)

KeyError: 0

In [75]:
# input -> (xi,yi)
# output -> (label) 2 class model 0,1

def perceptron(epochs,rate,input,target):
  w = np.zeros(input.shape[1]) # weights initialisation
  b = 0 # bias  initialisation
  for iter in range(0,epochs):
    for i in range(0,rows):
      x = np.array(input.iloc[i,:])
      y_pred = w.T@x + b
      y_actu = target.iloc[i]
      w += rate * (y_actu - y_pred) * x
      b += rate * (y_actu - y_pred)

  return w,b

In [76]:
w_new,b_new = perceptron(200,0.01,input,target)

<ipython-input-75-e420c6b535a3>:10: RuntimeWarning: overflow encountered in matmul
  y_pred = w.T@x + b
<ipython-input-75-e420c6b535a3>:12: RuntimeWarning: invalid value encountered in add
  w += rate * (y_actu - y_pred) * x
<ipython-input-75-e420c6b535a3>:13: RuntimeWarning: invalid value encountered in double_scalars
  b += rate * (y_actu - y_pred)


## TASK-B

# Data Generation

In [7]:
mean = np.zeros(6)
cov = np.identity(6)
data2 = np.random.multivariate_normal(mean, cov, size=(1000))

y1= 3x1+ 4x4+ 2x5+x6+ 3

y2= 2x1+ 3x2+ 5x3+ 7x4+ 4x5+ 8x6

y3=x2+x3+ 10

In [10]:
y1 = []
y2 = []
y3 = []
for i in range(0,1000):
  y1.append( 3*(data2[i][0]) +  4*(data2[i][3]) +  2*data2[i][4] + data2[i][5] + 3)
  y2.append( 2*(data2[i][0])+ 3*(data2[i][1])+ 5*(data2[i][2])+ 7*(data2[i][3])+ 4*(data2[i][4])+ 8*(data2[i][5]))
  y3.append( (data2[i][1]) + (data2[i][2]) + 10 )

In [11]:
x1 = data2[:,0]
x2 = data2[:,1]
x3 = data2[:,2]
x4 = data2[:,3]
x5 = data2[:,4]
x6 = data2[:,5]

In [12]:
dict = {'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'x5': x5, 'x6': x6, 'y1': y1, 'y2': y2, 'y3': y3 }
df = pd.DataFrame(dict)

In [13]:
df.shape

(1000, 9)

# Neural Network Model

In [23]:
import os
import torch
import torch.nn as nn
import torch.optim as optim

In [24]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [52]:
input_train = df.iloc[:800,:6]
target_train = df.iloc[:800,6:]
input_test = df.iloc[800:,0:6]
target_test = df.iloc[800:,6:]

In [64]:
X_train = torch.tensor(input_train.values, dtype=torch.float32)
y_train = torch.tensor(target_train.values, dtype=torch.float32).reshape(-1,3)
X_test = torch.tensor(input_test.values, dtype=torch.float32)
y_test = torch.tensor(target_test.values, dtype=torch.float32).reshape(-1,3)

In [65]:
# define the model
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(6, 10)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(10, 10)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(10, 3)
        self.act_output = nn.ReLU()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = x = self.act_output(self.output(x))
        return x

model = PimaClassifier()
print(model)

PimaClassifier(
  (hidden1): Linear(in_features=6, out_features=10, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=10, out_features=10, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=10, out_features=3, bias=True)
  (act_output): ReLU()
)


In [66]:
loss_fn = nn.MSELoss() #mean square error
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [71]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from sklearn.model_selection import train_test_split

In [73]:
# train the model
n_epochs = 100
batch_size = 64

for epoch in range(n_epochs):
    for i in range(0, len(X_train), batch_size):
        Xbatch = X_train[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y_train[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0 :
      print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 77.12981414794922
Finished epoch 10, latest loss 77.12892150878906
Finished epoch 20, latest loss 77.1285171508789
Finished epoch 30, latest loss 77.1281967163086
Finished epoch 40, latest loss 77.1279067993164
Finished epoch 50, latest loss 77.12769317626953
Finished epoch 60, latest loss 77.12757110595703
Finished epoch 70, latest loss 77.12744903564453
Finished epoch 80, latest loss 77.1272964477539
Finished epoch 90, latest loss 77.12711334228516


In [ ]:
#testing the model
